In [5]:
import numpy as np
import pandas as pd
import re

from data_util import create_dataset, gene_selection, remove_mean_cutoffs_tumor
from data_util import get_healthy_genes, differential_gene_expression_analysis, setup_gene_name_dict
from data_util import run_enrichr, remove_mean_cutoffs_healthy, get_healthy2_gene_list, get_healthy_tissue_gene_exp_df

from sklearn.preprocessing import StandardScaler
from scipy.stats import ttest_ind
from scipy.stats import norm

import gseapy as gp
import math
from matplotlib import pyplot as plt

# Read in data

###  Note that the loaded data has already been log_2(x+1) transformed from the raw values

In [14]:
cancer_type = 'BLCA'
cancer_origin = 'Bladder'
tumor_site1 = 'Lung'
tumor_site2 = 'Liver'
healthy_files_created = True

# List of cutoffs used in the analysis:
# 0) mean_cutoff_tumor_site1 = 1
# 1) mean_cutoff_tumor_site2 = 1
# 2) p_value_cutoff_tumor_site1 = 0.01
# 3) p_value_cutoff_tumor_site2 = 0.01
# 4) mean_cutoff_healthy2 = 4
# 5) p_value_cutoff_healthy2 = 0.01
cutoff_list = [1, 1, 0.01, 0.01, 4, 0.01]

In [3]:
filepath_new = '../mmc1-tony.xlsx'
data_new = pd.read_excel(filepath_new,index_col=0)

data_tumor_site1 = create_dataset(cancer_type = 'BLCA',
                                new_tumor_event_site = 'Lung',
                                data_new = data_new,
                                filepath_ge = '../BLCA/TCGA-BLCA.htseq_fpkm.tsv',
                                filepath_ph = '../BLCA/TCGA-BLCA.GDC_phenotype.tsv')

data_tumor_site2 = create_dataset(cancer_type = 'BLCA',
                                new_tumor_event_site = 'Liver',
                                data_new = data_new,
                                filepath_ge = '../BLCA/TCGA-BLCA.htseq_fpkm.tsv',
                                filepath_ph = '../BLCA/TCGA-BLCA.GDC_phenotype.tsv')

print('==========Raw Data==========')
print('{}-{} sample count: {}'.format(cancer_type, tumor_site1, data_tumor_site1.shape[0]))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site1, data_tumor_site1.shape[1]))

print('\n{}-{} sample count: {}'.format(cancer_type, tumor_site2, data_tumor_site2.shape[0]))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site2, data_tumor_site2.shape[1]))

==========Raw Data==========
BLCA-Lung sample count: 31
BLCA-Lung gene count: 60487

BLCA-Liver sample count: 12
BLCA-Liver gene count: 60487


In [13]:
# The pipeline requires two healthy gene expression datasets for each analysis

if healthy_files_created == False:

    sampleAttributesDS_path = '../GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt'
    gtexTCGA_path = '../TcgaTargetGtex_rsem_gene_fpkm'

    # Healthy tissue data to compare with the tumor data from the cancer origin site (e.g. BRCA_lung to healthy breast)
    healthy1_gene_exp_df = get_healthy_tissue_gene_exp_df(sample_attr_DS_path=sampleAttributesDS_path,
                                                      gtex_tcga_path=gtexTCGA_path,
                                                      tissue_str=cancer_origin)

    # Healthy tissue data to compare with the first metastatic site (e.g. BRCA_lung to healthy lung)
    healthy2_gene_exp_df = get_healthy_tissue_gene_exp_df(sample_attr_DS_path=sampleAttributesDS_path,
                                                      gtex_tcga_path=gtexTCGA_path,
                                                      tissue_str=tumor_site1)

    # Write healthy tissue data to csv
    healthy1_gene_exp_df.to_csv('../{}_gene_exp_healthy.csv'.format(cancer_origin))
    healthy2_gene_exp_df.to_csv('../{}_gene_exp_healthy.csv'.format(tumor_site1))

# Selection of Genes Related to Transporters and Enzymes

In [8]:
# Get names df - List of genes related to transporter and enzyme production
names_df = pd.read_csv('../ensembl_index.csv',index_col = 0)

# Get bladder-lung data
data_tumor_site1 = gene_selection(data_tumor_site1, names_df)

# Manually inspect to make sure all samples end in "01A" or "01B"
data_tumor_site1.drop('TCGA-GD-A2C5-11A', inplace=True) # Remove as sample ends in "11A"

# Get bladder-liver data
data_tumor_site2 = gene_selection(data_tumor_site2, names_df)

print('==========Post Gene Selection==========')
print('{}-{} sample count: {}'.format(cancer_type, tumor_site1, data_tumor_site1.shape[0]))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site1, data_tumor_site1.shape[1]))

print('\n{}-{} sample count: {}'.format(cancer_type, tumor_site2, data_tumor_site2.shape[0]))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site2, data_tumor_site2.shape[1]))    

==========Post Gene Selection==========
BLCA-Lung sample count: 30
BLCA-Lung gene count: 4192

BLCA-Liver sample count: 12
BLCA-Liver gene count: 4192


# Histograms and Mean Cutoffs of Tumor Samples

In [9]:
data_tumor_site1 = remove_mean_cutoffs_tumor(data_tumor_site1, cutoff = cutoff_list[0], plotting = False)
data_tumor_site2 = remove_mean_cutoffs_tumor(data_tumor_site2, cutoff = cutoff_list[1], plotting = False)

print('==========Post Mean Cutoffs==========')
print('{}-{} sample count: {}'.format(cancer_type, tumor_site1, data_tumor_site1.shape[0]))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site1, data_tumor_site1.shape[1]))

print('\n{}-{} sample count: {}'.format(cancer_type, tumor_site2, data_tumor_site2.shape[0]))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site2, data_tumor_site2.shape[1]))    

==========Post Mean Cutoffs==========
BLCA-Lung sample count: 30
BLCA-Lung gene count: 2959

BLCA-Liver sample count: 12
BLCA-Liver gene count: 2966


# Differential Gene Expression Analysis

In [15]:
filepath = '../Bladder_gene_exp_healthy.csv'
healthy1_trunc_df = get_healthy_genes(filepath)

tumor_site1_gene_list, _ = differential_gene_expression_analysis(data_tumor_site1, healthy1_trunc_df)
tumor_site2_gene_list, _ = differential_gene_expression_analysis(data_tumor_site2, healthy1_trunc_df)

print('==========Post Differential Gene Expression Analysis==========')
print('{}-{} gene count: {}'.format(cancer_type, tumor_site1, len(tumor_site1_gene_list)))
print('{}-{} gene count: {}'.format(cancer_type, tumor_site2, len(tumor_site2_gene_list)))

==========Post Differential Gene Expression Analysis==========
BLCA-Lung gene count: 234
BLCA-Liver gene count: 209


# Gene Set Enrichment Analysis

In [16]:
# Setup ensembl/name dictionaries
filepath = '../Ensembl_symbol_entrez.csv'
ensembl2gene, gene2ensembl = setup_gene_name_dict(filepath)

# Setup gene_sets libraries
gene_sets = ['GO_Biological_Process_2018', 'Reactome_2016']

# Run GSEA on BRCA-to-lung data
tumor_site1_gene_enr_list = [ensembl2gene[ensembl] for ensembl in tumor_site1_gene_list]
tumor_site1_enr = run_enrichr(gene_list = tumor_site1_gene_enr_list, gene_sets = gene_sets)
tumor_site1_enr_results = tumor_site1_enr.results

# Run GSEA on BRCA-to-liver data
tumor_site2_gene_enr_list = [ensembl2gene[ensembl] for ensembl in tumor_site2_gene_list]
tumor_site2_enr = run_enrichr(gene_list = tumor_site2_gene_enr_list, gene_sets = gene_sets)
tumor_site2_enr_results = tumor_site2_enr.results

# Get intersection of pathways for both metastatic sites
tumor_site1_pw = set(tumor_site1_enr_results[tumor_site1_enr_results['Adjusted P-value'] < cutoff_list[2]]['Term'])
tumor_site2_pw = set(tumor_site2_enr_results[tumor_site2_enr_results['Adjusted P-value'] < cutoff_list[3]]['Term'])

# Get difference between intersection of pathways and preferred metastatic site pathway
tumor_inter_pw = tumor_site1_pw.intersection(tumor_site2_pw)
tumor_site1_diff_pw = tumor_site1_pw.difference(tumor_inter_pw)

# Mean Cutoffs for Healthy Tissue Samples

In [ ]:
filepath = '../Lung_gene_exp_healthy.csv'
healthy2_trunc_df = get_healthy_genes(filepath)

healthy2_results_df = remove_mean_cutoffs_healthy(healthy2_trunc_df, cutoff = cutoff_list[4], plotting = False)

print('Pre-cutoff - healthy tissue gene count: {}'.format(healthy2_trunc_df.shape[1]))
print('Post-cutoff - healthy tissue gene count: {}'.format(healthy2_results_df.shape[1]))

# Gene Set Enrichment Analysis for Healthy Samples

In [ ]:
healthy2_gene_list, missing_id_list = get_healthy2_gene_list(healthy2_results_df)

# Run Enrichr on healthy2 gene list
healthy2_enr = run_enrichr(gene_list = healthy2_gene_list, gene_sets = gene_sets)
healthy2_enr_results = healthy2_enr.results

healthy2_pw = set(healthy2_enr_results[healthy2_enr_results['Adjusted P-value'] < cutoff_list[5]]['Term'])

# Get intersection between healthy2 and tumor_site1_diff
tumor_site1_diff_healthy2_inter_pw = tumor_site1_diff_pw.intersection(healthy2_pw)

# Print pre and post pathway counts
print('Initial significant pathway count: {}'.format(len(tumor_site1_diff_pw)))
print('Post-intersection pathway count: {}'.format(len(tumor_site1_diff_healthy2_inter_pw)))

In [ ]:
list(tumor_site1_diff_healthy2_inter_pw)

# K-Means Clustering

In [50]:
# gene2idx = {}
# idx2gene = {}

# for i, gene in enumerate(gene_list):
#     gene2idx[gene] = i
#     idx2gene[i] = gene

In [51]:
# from sklearn.cluster import KMeans
# from sklearn import preprocessing

# # Keep subset of genes that are effectively upregulated
# BRCA_lung_subset_df = BRCA_lung_trunc_df[gene_list]
# gene_array = BRCA_lung_subset_df.to_numpy()
# gene_array_norm = preprocessing.normalize(gene_array.T)

# def run_km(cluster_count_list, gene_list, verbose=False, plotting=False):
    
#     km_list = []

#     # gene_array

#     gene_list = np.array(gene_list)

#     for cluster_count in cluster_count_list:
    
#         km2 = KMeans(n_clusters=cluster_count,init='random').fit(gene_array_norm)

#         if verbose == True:

#             for i in range(cluster_count):
#                 print('----------Cluster {}----------'.format(i))
#                 print(gene_list[km2.labels_==i])

#         gene_list_km = []

#         for i in range(cluster_count):
#             gene_list_km.append(gene_list[km2.labels_==i])

#         gene_index_km = [item for sublist in gene_list_km for item in sublist]
#         gene_index_idx = [gene2idx[gene] for gene in gene_index_km]
        
#         if plotting == True:
        
#             plt.figure(figsize=(10,5))
#             df = np.corrcoef(gene_array.T[gene_index_idx])
#             plt.matshow(df, fignum=1)
#             # plt.xticks(range(len(df.columns)), df.columns)
#             # plt.yticks(range(len(df.columns)), df.columns)
#             plt.colorbar()
#             plt.show()
            
#         km_list.append(km2)
            
#     return km_list

In [52]:
# from sklearn.cluster import KMeans 
# from sklearn import metrics 
# from scipy.spatial.distance import cdist 
# import numpy as np 
# import matplotlib.pyplot as plt  

# distortions = [] 
# inertias = [] 
# mapping1 = {} 
# mapping2 = {} 
# K = range(1,20) 
  
# for k in K: 
#     #Building and fitting the model 
#     kmeanModel = KMeans(n_clusters=k).fit(gene_array_norm) 
#     kmeanModel.fit(gene_array_norm)     
      
#     distortions.append(sum(np.min(cdist(gene_array_norm, kmeanModel.cluster_centers_, 
#                       'euclidean'),axis=1)) / gene_array_norm.shape[0]) 
#     inertias.append(kmeanModel.inertia_) 
  
#     mapping1[k] = sum(np.min(cdist(gene_array_norm, kmeanModel.cluster_centers_, 
#                  'euclidean'),axis=1)) / gene_array_norm.shape[0] 
#     mapping2[k] = kmeanModel.inertia_ 

In [53]:
# for key,val in mapping1.items(): 
#     print(str(key)+' : '+str(val)) 

# plt.plot(K, distortions, 'bx-') 
# plt.xlabel('Values of K') 
# plt.ylabel('Distortion') 
# plt.title('The Elbow Method using Distortion') 
# plt.show() 

In [54]:
# for key,val in mapping2.items(): 
#     print(str(key)+' : '+str(val)) 

# plt.plot(K, inertias, 'bx-') 
# plt.xlabel('Values of K') 
# plt.ylabel('Inertia') 
# plt.title('The Elbow Method using Inertia') 
# plt.show()

In [55]:
# cluster_count_list = [4] * 20

# km_list = run_km(cluster_count_list,gene_list=gene_list,plotting=True)

In [ ]:
# labels = [int(x) for x in km_list[2].labels_]
# cluster_labels = np.unique(labels)
# gene_cluster_dict = {gene: label for label, gene in zip(labels, gene_list)}

In [ ]:
# def get_cluster_genes(cluster_id, labels, gene_list):
#     cluster_idx = np.array([label == cluster_id for label in labels])
#     gene_array = np.array(gene_list)
#     cluster_genes = gene_array[cluster_idx]        
    
#     return cluster_genes

# cluster_list = np.zeros(len(cluster_labels))
# cluster_id = 0
# labels = [int(x) for x in km_list[2].labels_]

# genes_cluster0 = get_cluster_genes(cluster_id, labels, gene_list)

In [ ]:
# BRCA_lung_subset_df

In [ ]:
# gp.get_library_name(database='Human')